In [1]:
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn import metrics
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

In [2]:
movie_df = pd.read_csv(r'data/data_regression.csv')

In [3]:
print(movie_df.shape)
movie_df.dtypes

(3852, 42)


imdb_id                       object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
movie_facebook_likes           int64
Biography                      int64
C

In [4]:
movie_df.head()

,imdb_id,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,Sci-Fi_Fantasy,Family_Animation,Action_Adventure,History_War,Others,director_rank,actor1_rank,actor2_rank,actor3_rank,gross_budget_ratio
0,tt0035423,James Mangold,125.0,123.0,446.0,821.0,Natasha Lyonne,20000.0,47095453.0,Comedy|Fantasy|Romance,...,1,0,0,0,0,8,6,3,4,0.981155
1,tt0080339,Jim Abrahams,134.0,88.0,104.0,318.0,Lloyd Bridges,628.0,83400000.0,Comedy,...,0,0,0,0,0,8,2,3,4,23.828571
2,tt0080453,Randal Kleiser,36.0,104.0,116.0,83.0,Christopher Atkins,1000.0,58853106.0,Adventure|Drama|Romance,...,0,0,1,0,0,7,3,3,3,13.078468
3,tt0080455,John Landis,125.0,148.0,644.0,326.0,Aretha Franklin,1000.0,54200000.0,Action|Comedy|Crime|Music,...,0,0,1,0,1,8,3,3,4,2.007407
4,tt0080487,Harold Ramis,71.0,98.0,11000.0,484.0,Rodney Dangerfield,13000.0,39800000.0,Comedy|Sport,...,0,0,0,0,1,7,7,3,5,6.633333


In [5]:
movie_df.describe()

,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,...,Sci-Fi_Fantasy,Family_Animation,Action_Adventure,History_War,Others,director_rank,actor1_rank,actor2_rank,actor3_rank,gross_budget_ratio
count,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3.852000e+03,3.852000e+03,3852.000000,3852.000000,3852.000000,...,3852.00000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000,3852.000000
mean,160.910436,109.049325,762.463396,730.276739,7563.897715,4.929950e+07,9.813550e+04,11170.345275,1.379283,313.869938,...,0.23702,0.119678,0.336708,0.063084,0.132918,7.076064,4.046729,3.432503,4.333333,5.690951
std,121.840785,21.549751,2988.626715,1805.214756,15431.666830,7.034357e+07,1.468241e+05,18845.309090,2.045882,397.309921,...,0.42531,0.324627,0.472646,0.243146,0.339530,1.090874,1.987470,1.347786,1.547315,129.951090
min,1.000000,34.000000,0.000000,0.000000,0.000000,6.180800e+01,4.000000e+01,0.000000,0.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000018
25%,72.750000,95.000000,10.000000,183.000000,726.250000,6.108100e+06,1.647275e+04,1833.750000,0.000000,98.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,7.000000,3.000000,3.000000,3.000000,0.422215
50%,132.000000,105.000000,58.000000,427.500000,1000.000000,2.626054e+07,4.757350e+04,3897.500000,1.000000,193.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,7.000000,3.000000,3.000000,4.000000,1.020191
75%,217.000000,119.000000,218.000000,681.000000,12000.000000,6.334179e+07,1.148290e+05,15823.750000,2.000000,375.000000,...,0.00000,0.000000,1.000000,0.000000,0.000000,8.000000,5.000000,4.000000,5.000000,2.088305
max,813.000000,330.000000,23000.000000,23000.000000,640000.000000,1.200264e+09,1.689764e+06,656730.000000,43.000000,5060.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,10.000000,10.000000,10.000000,10.000000,7194.485533


# IMDb rating prediction


## 1 Model 

For IMDB score we assume the following variables: duration, budget,gross, genre, rankings of the director and actors ,likes and face number in poster.

In [6]:
data1 = movie_df[['duration','budget','gross','Biography','Comedy','Crime','Drama','Romance',
                  'Mystery_Thriller_Horror','Sci-Fi_Fantasy','Family_Animation','Action_Adventure',
                  'History_War','Others','director_rank',
                 'actor1_rank','actor2_rank','actor3_rank'
                  ,'movie_facebook_likes','cast_total_facebook_likes','facenumber_in_poster']]

In [7]:
data1.head()

,duration,budget,gross,Biography,Comedy,Crime,Drama,Romance,Mystery_Thriller_Horror,Sci-Fi_Fantasy,...,Action_Adventure,History_War,Others,director_rank,actor1_rank,actor2_rank,actor3_rank,movie_facebook_likes,cast_total_facebook_likes,facenumber_in_poster
0,123.0,48000000.0,47095453.0,0,1,0,0,1,0,1,...,0,0,0,8,6,3,4,0,22209,1.0
1,88.0,3500000.0,83400000.0,0,1,0,0,0,0,0,...,0,0,0,8,2,3,4,16000,2726,0.0
2,104.0,4500000.0,58853106.0,0,0,0,1,1,0,0,...,1,0,0,7,3,3,3,0,1662,0.0
3,148.0,27000000.0,54200000.0,0,1,1,0,0,0,0,...,1,0,1,8,3,3,4,14000,2566,1.0
4,98.0,6000000.0,39800000.0,0,1,0,0,0,0,0,...,0,0,1,7,7,3,5,0,14921,3.0


In [8]:
y = movie_df['imdb_score']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data1, y, test_size=0.30)

MM = MinMaxScaler(feature_range=(0, 1), copy=True)
X_train_MM = MM.fit_transform(X_train) # standardize X_train
X_test_MM = MM.transform(X_test) # standardize X_test
X_train_scaled = pd.DataFrame(data=X_train_MM,  columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=X_test_MM,  columns=X_test.columns)

### linear regression

In [10]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred= lr.predict(X_test_scaled)
print("linear regression R2:", mean_squared_error(y_test, y_pred, multioutput='raw_values'))
print("linear regression MSE:", r2_score(y_test, y_pred))

linear regression R2: [0.35717232]
linear regression MSE: 0.6704959187027792


In [11]:
df0 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df0

,Actual,Predicted
241,5.3,5.492815
3840,6.4,6.139228
512,5.3,5.644742
388,6.1,6.509674
1364,8.1,7.684072
...,...,...
3514,7.6,7.462267
37,7.2,6.986822
437,7.1,7.183917
140,4.2,4.058681


### ridge

In [12]:
parameters = {'alpha': np.concatenate((np.arange(0.1,2,0.1), np.arange(2, 5, 0.5), np.arange(5, 25, 1)))}
ridge = linear_model.Ridge()
gridridge = GridSearchCV(ridge, parameters, scoring ='r2')
gridridge.fit(X_train_scaled, y_train)

y_pred = gridridge.predict(X_test_scaled)

print("ridge best parameters:", gridridge.best_params_)
print("ridge score:", gridridge.score(X_test_scaled, y_test))
print("ridge MSE:", mean_squared_error(y_test, gridridge.predict(X_test_scaled)))
print("ridge best estimator coef:", gridridge.best_estimator_.coef_)

ridge best parameters: {'alpha': 0.6}
ridge score: 0.6698928027119839
ridge MSE: 0.3578260753542666
ridge best estimator coef: [ 0.52388815 -0.243096    1.16256404  0.16272857 -0.04827998  0.07849334
  0.19320887 -0.04953255 -0.10484221 -0.05106441  0.02928414 -0.08610556
  0.04194851  0.03061913  5.3569568   0.11461175 -0.17449234 -0.29288886
  1.09916931 -0.00677609 -0.42946571]


In [13]:
df1 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df1

,Actual,Predicted
241,5.3,5.500619
3840,6.4,6.137831
512,5.3,5.653085
388,6.1,6.511269
1364,8.1,7.665244
...,...,...
3514,7.6,7.437589
37,7.2,6.982423
437,7.1,7.183496
140,4.2,4.080623


### lasso


In [14]:
parameters = {'alpha': np.concatenate((np.arange(0.01,2,0.1), np.arange(2, 5, 0.5), np.arange(5, 25, 1)))}
lasso = linear_model.Lasso()
gridlasso = GridSearchCV(lasso, parameters, scoring ='r2')
gridlasso.fit(X_train_scaled, y_train)

y_pred = gridlasso.predict(X_test_scaled)

print("lasso best parameters:", gridlasso.best_params_)
print("lasso score:", gridlasso.score(X_test_scaled, y_test))
print("lasso MSE:", mean_squared_error(y_test, gridlasso.predict(X_test_scaled)))
print("lasso best estimator coef:", gridlasso.best_estimator_.coef_)

lasso best parameters: {'alpha': 0.01}
lasso score: 0.6519333379458621
lasso MSE: 0.3772935842287177
lasso best estimator coef: [ 0.         -0.          0.          0.08566186 -0.05074962  0.
  0.20772411 -0.         -0.05638249 -0.         -0.         -0.
  0.          0.          5.27389509  0.         -0.         -0.
  0.          0.         -0.        ]


In [15]:
df2 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df2

,Actual,Predicted
241,5.3,5.615530
3840,6.4,6.274767
512,5.3,5.766872
388,6.1,6.482491
1364,8.1,7.643990
...,...,...
3514,7.6,6.928371
37,7.2,7.141728
437,7.1,7.192477
140,4.2,4.240674


### elastic net

In [16]:
parameters = {'alpha': np.concatenate((np.arange(0.01,2,0.1), np.arange(2, 5, 0.5), np.arange(5, 25, 1)))}
en = lasso = linear_model.ElasticNet()
griden = GridSearchCV(en, parameters, scoring ='r2')
griden.fit(X_train_scaled, y_train)

y_pred = griden.predict(X_test_scaled)

print("Elastic Net best parameters:", griden.best_params_)
print("Elastic Net score:", griden.score(X_test_scaled, y_test))
print("Elastic Net MSE:", mean_squared_error(y_test, griden.predict(X_test_scaled)))
print("Elastic Net best estimator coef:", griden.best_estimator_.coef_)

Elastic Net best parameters: {'alpha': 0.01}
Elastic Net score: 0.6214229049390934
Elastic Net MSE: 0.410365957657298
Elastic Net best estimator coef: [ 1.58126026e-01  0.00000000e+00  1.42210924e-01  1.57595825e-01
 -1.13877156e-01  4.39499322e-02  2.67523422e-01 -3.55309798e-02
 -1.25687177e-01 -3.31430442e-03 -0.00000000e+00 -2.33617416e-02
  3.74662318e-02  0.00000000e+00  4.16117641e+00  8.10529440e-02
 -0.00000000e+00 -2.77341466e-02  2.98228649e-01  0.00000000e+00
 -0.00000000e+00]


In [17]:
df3 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df3

,Actual,Predicted
241,5.3,5.677270
3840,6.4,6.195193
512,5.3,5.867269
388,6.1,6.504837
1364,8.1,7.374992
...,...,...
3514,7.6,6.867053
37,7.2,7.005254
437,7.1,7.123703
140,4.2,4.507383


## 2 Model

Here we exclude genre from 1 Model

In [18]:
data2 = movie_df[['duration','budget','gross','director_rank',
                 'actor1_rank','actor2_rank','actor3_rank'
                  ,'movie_facebook_likes','cast_total_facebook_likes','facenumber_in_poster']]

In [19]:
y = movie_df['imdb_score']
X_train, X_test, y_train, y_test = train_test_split(data2, y, test_size=0.30)

MM = MinMaxScaler(feature_range=(0, 1), copy=True)
X_train_MM = MM.fit_transform(X_train) # standardize X_train
X_test_MM = MM.transform(X_test) # standardize X_test
X_train_scaled = pd.DataFrame(data=X_train_MM,  columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=X_test_MM,  columns=X_test.columns)

### linear regression

In [20]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred= lr.predict(X_test_scaled)
print("linear regression R2:", mean_squared_error(y_test, y_pred, multioutput='raw_values'))
print("linear regression MSE:", r2_score(y_test, y_pred))

linear regression R2: [0.42179587]
linear regression MSE: 0.6356143552469395


In [21]:
df4 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df4

,Actual,Predicted
1025,6.0,5.548631
2562,7.1,6.345528
981,5.7,6.389580
3270,7.2,7.910294
1878,6.8,6.356936
...,...,...
84,7.4,7.070761
2720,6.5,6.159642
1012,8.0,7.158364
2071,5.4,6.119834


### ridge

In [22]:
parameters = {'alpha': np.concatenate((np.arange(0.1,2,0.1), np.arange(2, 5, 0.5), np.arange(5, 25, 1)))}
ridge = linear_model.Ridge()
gridridge = GridSearchCV(ridge, parameters, scoring ='r2')
gridridge.fit(X_train_scaled, y_train)

y_pred = gridridge.predict(X_test_scaled)

print("ridge best parameters:", gridridge.best_params_)
print("ridge score:", gridridge.score(X_test_scaled, y_test))
print("ridge MSE:", mean_squared_error(y_test, gridridge.predict(X_test_scaled)))
print("ridge best estimator coef:", gridridge.best_estimator_.coef_)

ridge best parameters: {'alpha': 0.6}
ridge score: 0.6348118904448636
ridge MSE: 0.4227247705546308
ridge best estimator coef: [ 0.94333671 -0.44139964  0.28493679  5.57548024  0.14134512 -0.12088307
 -0.39970264  1.00974835  0.05495699 -0.54065241]


In [23]:
df5 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df5

,Actual,Predicted
1025,6.0,5.557854
2562,7.1,6.345244
981,5.7,6.390651
3270,7.2,7.892269
1878,6.8,6.354389
...,...,...
84,7.4,7.058764
2720,6.5,6.157566
1012,8.0,7.148134
2071,5.4,6.122254


### lasso

In [24]:
parameters = {'alpha': np.concatenate((np.arange(0.01,2,0.1), np.arange(2, 5, 0.5), np.arange(5, 25, 1)))}
lasso = linear_model.Lasso()
gridlasso = GridSearchCV(lasso, parameters, scoring ='r2')
gridlasso.fit(X_train_scaled, y_train)

y_pred = gridlasso.predict(X_test_scaled)

print("lasso best parameters:", gridlasso.best_params_)
print("lasso score:", gridlasso.score(X_test_scaled, y_test))
print("lasso MSE:", mean_squared_error(y_test, gridlasso.predict(X_test_scaled)))
print("lasso best estimator coef:", gridlasso.best_estimator_.coef_)

lasso best parameters: {'alpha': 0.01}
lasso score: 0.6210672455374627
lasso MSE: 0.4386349322297023
lasso best estimator coef: [ 0.       -0.        0.        5.507352  0.       -0.       -0.
  0.        0.       -0.      ]


In [25]:
df6 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df6

,Actual,Predicted
1025,6.0,5.702645
2562,7.1,6.391064
981,5.7,6.391064
3270,7.2,7.767902
1878,6.8,6.391064
...,...,...
84,7.4,7.079483
2720,6.5,6.391064
1012,8.0,7.079483
2071,5.4,6.391064


### elastic net

In [26]:
parameters = {'alpha': np.concatenate((np.arange(0.01,2,0.1), np.arange(2, 5, 0.5), np.arange(5, 25, 1)))}
en = linear_model.ElasticNet()
gridlasso = GridSearchCV(en, parameters, scoring ='r2')
gridlasso.fit(X_train_scaled, y_train)

y_pred = gridlasso.predict(X_test_scaled)

print("lasso best parameters:", gridlasso.best_params_)
print("lasso score:", gridlasso.score(X_test_scaled, y_test))
print("lasso MSE:", mean_squared_error(y_test, gridlasso.predict(X_test_scaled)))
print("lasso best estimator coef:", gridlasso.best_estimator_.coef_)

lasso best parameters: {'alpha': 0.01}
lasso score: 0.5892668634370936
lasso MSE: 0.47544557549873934
lasso best estimator coef: [ 0.47890129  0.          0.10417754  4.41413532  0.11178754 -0.
 -0.14765653  0.2475415   0.         -0.        ]


In [27]:
df7 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df7

,Actual,Predicted
1025,6.0,5.810487
2562,7.1,6.374125
981,5.7,6.392811
3270,7.2,7.557663
1878,6.8,6.354624
...,...,...
84,7.4,6.914931
2720,6.5,6.257978
1012,8.0,6.984209
2071,5.4,6.323641
